In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns  
import json  
from pathlib import Path  
import plotly.express as px  
import plotly.graph_objects as go  
from plotly.subplots import make_subplots  
import glob  

# Set style for better visualization  
sns.set_palette("husl")   # Set Seaborn color palette

###################  
# Data Loading   
###################

In [2]:
# Load algorithm comparison data  
with open('C:/Uni/introb/traditional_algorithms.json', 'r') as f:  
    trad_data = json.load(f)  

with open('C:/Uni/introb/rl_algorithms.json', 'r') as f:  
    rl_data = json.load(f)  

# Function to load training data for RL algorithms  
def load_training_data(algorithm):  
    """  
    Load all training runs for a specific algorithm  

    Args:  
        algorithm: 'ql' or 'sarsa'  
    Returns:  
        Dictionary with run data indexed by timestamp  
    """  
    base_path = Path('C:/Uni/introb/training_data')  
    training_runs = {}  

    pattern = f"{algorithm}_training_*"  
    run_dirs = list(base_path.glob(pattern))  

    for run_dir in run_dirs:  
        timestamp = run_dir.name.split('_')[-2] + '_' + run_dir.name.split('_')[-1]  
        with open(run_dir / 'final_training_data.json', 'r') as f:  
            training_runs[timestamp] = json.load(f)  

    return training_runs  

# Load training data for both RL algorithms  
ql_training_data = load_training_data('ql')  
sarsa_training_data = load_training_data('sarsa')  

# Print data overview  
print("Data Overview:")  
print("-------------")  
print(f"Traditional Algorithms Maps: {list(trad_data.keys())}")  
print(f"Traditional Algorithms per Map: {list(trad_data['DIAGONAL'].keys())}")  
print(f"RL Algorithms Maps: {list(rl_data.keys())}")  
print(f"RL Algorithms: {list(rl_data['DIAGONAL'].keys())}")  
print(f"Q-Learning training runs loaded: {len(ql_training_data)}")  
print(f"SARSA training runs loaded: {len(sarsa_training_data)}")

Data Overview:
-------------
Traditional Algorithms Maps: ['DIAGONAL', 'SNAKE', 'OPEN', 'BOTTLENECK']
Traditional Algorithms per Map: ['ASTAR', 'DIJKSTRA', 'GBFS', 'BFS']
RL Algorithms Maps: ['DIAGONAL']
RL Algorithms: ['QL', 'SARSA']
Q-Learning training runs loaded: 4
SARSA training runs loaded: 4


###################  
# 1. Traditional Algorithms Analysis  
###################  

In [3]:
def analyze_traditional_algorithms(data):  
    """Analyze performance of traditional pathfinding algorithms"""  
    # Create comparison metrics across maps  
    metrics_df = pd.DataFrame([  
        {  
            'Algorithm': algo,  
            'Map': map_name,  
            **{k:v for k,v in stats.items() if k in [  
                'avg_path_length', 'avg_completion_time',   
                'avg_nodes_explored', 'avg_score',  
                'avg_exploration_efficiency'  
            ]}  
        }  
        for map_name, algos in data.items()  
        for algo, stats in algos.items()  
        if algo in ['ASTAR', 'DIJKSTRA', 'GBFS', 'BFS']  
    ])  

    # Visualization 1: Performance Metrics by Algorithm  
    fig = make_subplots(rows=2, cols=2, subplot_titles=(  
        'Average Path Length', 'Average Completion Time',  
        'Nodes Explored', 'Exploration Efficiency'  
    ))  

    metrics = ['avg_path_length', 'avg_completion_time',   
               'avg_nodes_explored', 'avg_exploration_efficiency']  

    for i, metric in enumerate(metrics):  
        row = i // 2 + 1  
        col = i % 2 + 1  

        fig.add_trace(  
            go.Bar(  
                x=metrics_df['Algorithm'].unique(),  
                y=metrics_df.groupby('Algorithm')[metric].mean(),  
                name=metric  
            ),  
            row=row, col=col  
        )  

    fig.update_layout(height=800, title_text="Traditional Algorithm Performance Comparison")  
    fig.show()  

    # Visualization 2: Algorithm Performance by Map Type  
    fig = px.scatter(metrics_df, x='avg_completion_time', y='avg_score',  
                    size='avg_nodes_explored', color='Algorithm',  
                    facet_col='Map', hover_data=['avg_exploration_efficiency'],  
                    title='Traditional Algorithm Performance across Different Maps')  
    fig.show()  

    return metrics_df

###################  
# 2. Reinforcement Learning Analysis  
###################

In [4]:
def analyze_rl_training(training_data, algorithm_name):  
    """Analyze training progress of RL algorithms"""  
    fig = make_subplots(  
        rows=2, cols=2,  
        subplot_titles=(  
            'Episode Rewards Over Time',  
            'Path Lengths Over Time',  
            'Exploration Rates',  
            'Nodes Explored per Episode'  
        )  
    )  

    for timestamp, run_data in training_data.items():  
        # Add episode rewards  
        fig.add_trace(  
            go.Scatter(  
                y=run_data['episode_rewards'],  
                name=f'Run {timestamp}',  
                showlegend=True,  
                mode='lines',  
                line=dict(width=1)  
            ),  
            row=1, col=1  
        )  

        # Add path lengths  
        fig.add_trace(  
            go.Scatter(  
                y=run_data['episode_lengths'],  
                name=f'Run {timestamp}',  
                showlegend=False,  
                mode='lines',  
                line=dict(width=1)  
            ),  
            row=1, col=2  
        )  

        # Add exploration rates  
        fig.add_trace(  
            go.Scatter(  
                y=run_data['exploration_rates'],  
                name=f'Run {timestamp}',  
                showlegend=False,  
                mode='lines',  
                line=dict(width=1)  
            ),  
            row=2, col=1  
        )  

        # Add nodes explored  
        fig.add_trace(  
            go.Scatter(  
                y=run_data['nodes_explored_per_episode'],  
                name=f'Run {timestamp}',  
                showlegend=False,  
                mode='lines',  
                line=dict(width=1)  
            ),  
            row=2, col=2  
        )  

    fig.update_layout(  
        height=800,  
        title_text=f"{algorithm_name} Training Analysis",  
        template="plotly_white"  
    )  

    fig.update_xaxes(title_text="Episode", row=2, col=1)  
    fig.update_xaxes(title_text="Episode", row=2, col=2)  
    fig.update_yaxes(title_text="Reward", row=1, col=1)  
    fig.update_yaxes(title_text="Path Length", row=1, col=2)  
    fig.update_yaxes(title_text="Exploration Rate", row=2, col=1)  
    fig.update_yaxes(title_text="Nodes Explored", row=2, col=2)  

    fig.show()  

    # Calculate summary statistics  
    summary_stats = {  
        'avg_final_reward': np.mean([list(run['episode_rewards'])[-1]   
                                   for run in training_data.values()]),  
        'avg_final_path_length': np.mean([list(run['episode_lengths'])[-1]   
                                        for run in training_data.values()]),  
        'avg_nodes_explored': np.mean([np.mean(run['nodes_explored_per_episode'])   
                                     for run in training_data.values()]),  
        'training_time': np.mean([run['training_summary']['total_training_time']   
                                for run in training_data.values()])  
    }  

    return summary_stats  

def analyze_rl_performance(algo_data):  
    """Analyze final performance of RL algorithms"""  
    rl_metrics_df = pd.DataFrame([  
        {  
            'Algorithm': algo,  
            'Map': map_name,  
            **{k:v for k,v in stats.items() if k in [  
                'avg_path_length', 'avg_completion_time',   
                'avg_nodes_explored', 'avg_score',  
                'avg_exploration_efficiency'  
            ]}  
        }  
        for map_name, algos in algo_data.items()  
        for algo, stats in algos.items()  
        if algo in ['QL', 'SARSA']  
    ])  

    # Visualization: RL Performance Comparison  
    fig = make_subplots(rows=2, cols=2, subplot_titles=(  
        'Average Path Length', 'Average Completion Time',  
        'Nodes Explored', 'Exploration Efficiency'  
    ))  

    metrics = ['avg_path_length', 'avg_completion_time',   
               'avg_nodes_explored', 'avg_exploration_efficiency']  

    for i, metric in enumerate(metrics):  
        row = i // 2 + 1  
        col = i % 2 + 1  

        fig.add_trace(  
            go.Bar(  
                x=rl_metrics_df['Algorithm'].unique(),  
                y=rl_metrics_df.groupby('Algorithm')[metric].mean(),  
                name=metric  
            ),  
            row=row, col=col  
        )  

    fig.update_layout(height=800, title_text="RL Algorithm Performance Comparison")  
    fig.show()  

    return rl_metrics_df

###################  
# 3. Comparative Analysis  
###################

In [5]:
def compare_all_algorithms(trad_df, rl_df):  
    """Compare all algorithms across different metrics"""  
    combined_df = pd.concat([trad_df, rl_df])  

    # Normalize metrics for radar chart  
    metrics = ['avg_path_length', 'avg_completion_time',   
              'avg_exploration_efficiency', 'avg_score']  

    normalized_df = combined_df.copy()  
    for metric in metrics:  
        normalized_df[metric] = (combined_df[metric] - combined_df[metric].min()) / (combined_df[metric].max() - combined_df[metric].min())  

    # Create radar chart  
    fig = go.Figure()  

    for algo in normalized_df['Algorithm'].unique():  
        algo_data = normalized_df[normalized_df['Algorithm'] == algo]  

        fig.add_trace(go.Scatterpolar(  
            r=[algo_data[metric].mean() for metric in metrics],  
            theta=metrics,  
            name=algo,  
            fill='toself'  
        ))  

    fig.update_layout(  
        polar=dict(radialaxis=dict(visible=True, range=[0, 1])),  
        showlegend=True,  
        title="Algorithm Comparison - Normalized Metrics"  
    )  
    fig.show()  

    return normalized_df

###################  
# 4. Summary and Conclusions  
###################

In [6]:
def generate_summary(normalized_df, ql_stats, sarsa_stats):
    """Generate final summary and recommendations"""
    # Separate traditional and RL algorithms
    trad_df = normalized_df[normalized_df['Algorithm'].isin(['ASTAR', 'DIJKSTRA', 'GBFS', 'BFS'])]
    rl_df = normalized_df[normalized_df['Algorithm'].isin(['QL', 'SARSA'])]

    # Calculate mean performance for each metric and algorithm
    metrics = {
        'Path Length': 'avg_path_length',
        'Execution Time': 'avg_completion_time',
        'Exploration Efficiency': 'avg_exploration_efficiency',
        'Overall Score': 'avg_score'
    }

    summary_data = []
    for metric_name, metric_col in metrics.items():
        best_trad = trad_df.groupby('Algorithm')[metric_col].mean()
        best_rl = rl_df.groupby('Algorithm')[metric_col].mean()

        # For time and path length, lower is better
        if metric_col in ['avg_path_length', 'avg_completion_time']:
            best_trad_algo = best_trad.idxmin()
            best_trad_value = best_trad.min()
            best_rl_algo = best_rl.idxmin()
            best_rl_value = best_rl.min()
            diff_percent = ((best_trad_value - best_rl_value) / best_trad_value) * 100
        else:
            best_trad_algo = best_trad.idxmax()
            best_trad_value = best_trad.max()
            best_rl_algo = best_rl.idxmax()
            best_rl_value = best_rl.max()
            diff_percent = ((best_rl_value - best_trad_value) / best_trad_value) * 100

        # Scale the difference for visualization (cap at ±50% for display)
        display_diff = np.clip(diff_percent, -50, 50)

        summary_data.append({
            'Metric': metric_name,
            'Traditional': best_trad_algo,
            'RL': best_rl_algo,
            'Difference': diff_percent,  # Keep original difference for printing
            'Trad_Value': 1,
            'RL_Value': 1 + (display_diff / 100)  # Use scaled difference for visualization
        })

    # Create visualization
    fig = go.Figure()

    categories = [d['Metric'] for d in summary_data]
    trad_algos = [d['Traditional'] for d in summary_data]
    rl_algos = [f"{d['RL']}" for d in summary_data]
    trad_values = [d['Trad_Value'] for d in summary_data]
    rl_values = [d['RL_Value'] for d in summary_data]

    fig.add_trace(go.Bar(
        name='Traditional',
        x=categories,
        y=trad_values,
        text=trad_algos,
        textposition='inside',
        marker_color='rgba(58, 71, 80, 0.6)',
        width=0.4
    ))

    fig.add_trace(go.Bar(
        name='Reinforcement Learning',
        x=categories,
        y=rl_values,
        text=rl_algos,
        textposition='inside',
        marker_color='rgba(246, 78, 139, 0.6)',
        width=0.4
    ))

    fig.update_layout(
        title={
            'text': "Best Performing Algorithms by Category",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        barmode='group',
        showlegend=True,
        plot_bgcolor='white',
        height=400,
        yaxis={'showticklabels': False, 'showgrid': False},
        xaxis={'showgrid': False},
        legend={
            'orientation': 'h',
            'yanchor': 'bottom',
            'y': 1.02,
            'xanchor': 'right',
            'x': 1
        }
    )

    fig.show()

    # Print detailed insights with actual differences
    print("\nDetailed Analysis:")
    print("----------------")
    for d in summary_data:
        print(f"\n{d['Metric']}:")
        print(f"  Best Traditional: {d['Traditional']}")
        print(f"  Best RL: {d['RL']}")
        print(f"  Difference: {d['Difference']:.1f}%")
        if d['Difference'] > 0:
            print("  RL performs BETTER")
        else:
            print("  Traditional performs BETTER")

    print("\nRL Training Times:")
    print(f"Q-Learning: {ql_stats['training_time']:.2f} seconds")
    print(f"SARSA: {sarsa_stats['training_time']:.2f} seconds")

    return summary_data

###################  
# Main Analysis  
###################

In [7]:
# Run traditional algorithm analysis
trad_metrics = analyze_traditional_algorithms(trad_data)

# Run RL training analysis
ql_stats = analyze_rl_training(ql_training_data, "Q-Learning")
sarsa_stats = analyze_rl_training(sarsa_training_data, "SARSA")

# Run RL performance analysis
rl_metrics = analyze_rl_performance(rl_data)

# Compare all algorithms
normalized_metrics = compare_all_algorithms(trad_metrics, rl_metrics)

# Generate final summary
summary = generate_summary(normalized_metrics, ql_stats, sarsa_stats)


Detailed Analysis:
----------------

Path Length:
  Best Traditional: ASTAR
  Best RL: SARSA
  Difference: 9.7%
  RL performs BETTER

Execution Time:
  Best Traditional: BFS
  Best RL: SARSA
  Difference: 16.9%
  RL performs BETTER

Exploration Efficiency:
  Best Traditional: GBFS
  Best RL: SARSA
  Difference: 1742.7%
  RL performs BETTER

Overall Score:
  Best Traditional: GBFS
  Best RL: SARSA
  Difference: 162.0%
  RL performs BETTER

RL Training Times:
Q-Learning: 58.97 seconds
SARSA: 31.10 seconds
